# Drug Interaction and Enzyme Kinetics

In this workbook we will introduce *Python* commands for importing and manipulating data. then we will curve fit and plot using what we learned in the previous tutorial.



In [ ]:
####################################
### Install UNCERTAINTIES package 
####################################

!pip install uncertainties  # to install in Colab. Add it back to avoid installing again and again.

####################################
### Load tools 
####################################

!wget https://raw.githubusercontent.com/blinkletter/3530Biochem/main/T04/T4Tools.py
import T4Tools

############################################
###  Import packages and set up variables
############################################

from os import system                    ## import tools to access the file sytsem directly
from scipy.optimize import curve_fit
import numpy as np                       ## import the tools of NumPy but use a shorter name
import pandas as pd                      ## import tools for data handling
from matplotlib import pyplot as plt
import requests                          ## tools to access web content

github_data_location= "https://raw.githubusercontent.com/blinkletter/3530Biochem/main/T04/data/"

####################################
### Import versions of NumPy and Math that use uncertainty 
####################################

import uncertainties as un
from uncertainties import unumpy as unp
from uncertainties import umath as um


## Setting Up the Data

The code below will print out the data file that we are using as text. Examine it and decide your next steps.

Choose from one of the following data files and modify the code below to access it...

- Data-0.csv
- Data-1.csv
- Data-2.csv
- Data-3.csv
- Data-4.csv
- Data-5.csv
- Data-6.csv
- Data-7.csv
- Data-8.csv
- Data-9.csv


## Example Analysis

In the following example I will obtain the $V_{max}$ and $K_M$ values in the case of warfarin with cyp450 and again with the presence of a potential inhibitor.

First let us examine the raw text data file with our eyes...

In [ ]:
########################
## Triclofenac B
########################

############################################
### Identify the data file and examine it
############################################

data_file = "Data-B.csv"       ### You will need to change this name

file_url = github_data_location + data_file   ### The data is on a web site so I add the web site to the file name here

page = requests.get(file_url)  ### pull in a file from the web (github in this case)
print(page.text)               ### print out the text contents of the web page file.


## Load the Data

We see that the data is organized in a format.  There are comments describing conditions and information about the format. This will enable you to interpret the data. There are also lines of data organized in columns. 

We will import this text fime into a data structure called a 'dataframe'. This is the *Python* version of a spreadsheet. The dataframe has tools built into it for manipulating the data. If you examine the code at the very end of the spreadsheet you will see examples of this.

Below we us the Pandas package (loaded in as 'pd') and the Pandas.read_csv() function to parse the data file and create the dataframe object. The dataframe is then printed out for your inspection.

Note that I manually entered some data that was in the comments, such as the concentration of the potential inhibitor and the enzyme concentration. You can use this data later to calculate the dissociation conatnat for inhibition, $K_i$.

In [ ]:
#######################
## Facts from data file comments
#######################

conc_enzyme = 1 ## nanomole/L 
conc_inh = 2    ## millimole/L
name_inh = "triclofenac B"
name_substrate = "warfarin"

################################################################################
### Read data set. The fields are separated by commas; comments are ignored 
################################################################################

data = pd.read_csv(file_url, 
                 delimiter = ",", 
                 skipinitialspace = True,
                 header = None, 
                 index_col = None, 
                 comment = "#") 
display(data)


## Edit the DataFrame

The dataframe has two sets of data in it.  the code below will separate the two chuncks of data and create two new dataframes. You will need to change the numbers in the slices if your data has a different number of lines that the set presented here.

The final dataframes are a seris of columns. there is a column for the time of measurements and columns for the amount of product formed at each time point. Each column is the result of an experiment with a different concentration of the substrate and these concentration values are listed at the top of each column. In a Pandas dataframe the 'header' is the title of each column. I can access each column using the header value. You can see examples of this in the code at the end of this notebook.

In [ ]:
################################################################################
### Separate the two groups of data and set up the dataframe for convenient access 
################################################################################

df = data.copy()           ### copy the dataframe to a generic name so I can easily reuse the code below
df = df[0:9]               ### slice rows 0 to 8 into a new dataframe

df_no_inh = T4Tools.trim_and_format_data(df)  ### will strip away unwanted rows and columns and set up column headers and index column


df = data.copy()           ### copy the dataframe again
df = df[9:]                ### slice rows 9 to the end into a new dataframe

df_inh = T4Tools.trim_and_format_data(df)     ### will strip away unwanted rows and columns and set up column headers and index column

################################################################################
### We now have dataframes with the data in rows labeled by the time points
### and each column labelled by the substrate concentration
################################################################################

print("Data for warfarin only no inhibitor")
display(df_no_inh)

print("Data for warfarin with drup -- potential inhibitor")
display(df_inh)


### Calculate Rates at Each Concentration

The experiment will enable us to calculate the reaction rate at a given concentration by taking the slope of the plot of concentration vs. time for the appearance of product. The function 'get_slopes_and_plot_them_all' will take a dataframe, curve fit each column agains time to get the slopes, plot all data and slopes, and finaly return the list of slopes and a list of the concentrations of the substrate.

You now have a set of rates and a set of concentrations.

In [ ]:
################################################################################
### Take a dataframe of product concentrations vs time at different substrate
### concentrations and get slopes of each experiment and plot them all
### on a single graph
################################################################################

concs, rates = T4Tools.get_slopes_and_plot_them_all(df_no_inh.copy(),
                                            title = "Change in concentration vs. Time", 
                                            xaxis = "$t /min$", 
                                            yaxis = "$\Delta [P] / \mu M$")

print(concs)
display(rates)

## Michaelis-Menten Plot

We will use a slightly modified vesrion of the MM_Plot() function from the previous tutorial.  This one has added error bars to the plot and uses the errors in the line fit (points with large errors are weighted less in the curve_fit algorithm).






In [ ]:
x = concs 
y = rates

Vmax, KM = T4Tools.MM_Plot(x, y)

print(f"Vmax = {Vmax:0.2f}\nKm = {KM:0.2f}")

## Lineweaver-Burke

We all hate it, but it is the one form of linearization that your textbook discusses so we will present it here. The Linear_Plot function from the previous tutorial has been modified to use errors as well. We will get the slope and intercept and calculate $V_{max}$ and $K_M$.

In [ ]:
x = 1 / concs 
y = 1 / rates    

slope, intercept = T4Tools.Linear_Plot(x, y,
                               title ="Lineweaver Something or Other", 
                               xaxis = "1 / x-axis", 
                               yaxis = "$1\\, /\\, \\nu$")

print(f"Slope = {slope:0.2f}\nIntercept = {intercept:0.3f}\n")  ### An exampe of the '\n' to make a new line

Vmax = 1 / intercept
KM = slope * Vmax

print(f"Vmax = {Vmax:0.2f}\nKm = {KM:0.2f}")

## You are Half Done

Now record the values you have calculated and download any plots you wish to present in your report. Then repeat the above with the other dataframe for the inhibited experiment. You will now have all the data that you need to finish the assignment.